# Test NIP-11 Relay Information Document

Interactive notebook for testing the `Nip11` model:
- Fetch from real relays (clearnet and overlay networks)
- Data access via `metadata.data["key"]`
- Proxy support for Tor/I2P/Loki relays
- Parsing and validation
- Error handling

In [4]:
import sys
sys.path.insert(0, "../src")

import asyncio
import json
from models.nip11 import Nip11, Nip11FetchError
from models.relay import Relay
from models.metadata import Metadata

print("Imports loaded successfully")

Imports loaded successfully


## 1. Basic Fetch from Clearnet Relay

In [5]:
# Test fetch from a well-known relay
relay = Relay("wss://relay.damus.io")
print(f"Relay: {relay.url}")
print(f"Network: {relay.network}")
print(f"Scheme: {relay.scheme}")
print(f"Host: {relay.host}")

Relay: wss://relay.damus.io
Network: clearnet
Scheme: wss
Host: relay.damus.io


In [6]:
# Fetch NIP-11 (no proxy needed for clearnet)
nip11 = await Nip11.fetch(relay)
print(f"Fetch successful: {nip11 is not None}")
print(f"Type: {type(nip11).__name__}")

Fetch successful: True
Type: Nip11


## 2. Data Access via metadata.data

In [7]:
# Access data via metadata.data dict
data = nip11.metadata.data

print("=== Base Fields ===")
print(f"name: {data['name']}")
print(f"description: {data['description']}")
print(f"pubkey: {data['pubkey']}")
print(f"contact: {data['contact']}")
print(f"software: {data['software']}")
print(f"version: {data['version']}")

=== Base Fields ===
name: damus.io
description: Damus strfry relay
pubkey: 32e1827635450ebb3c5a7d12c1f8e7b2b514439ac10a67eef3d9fd9c5c68e245
contact: jb55@jb55.com
software: git+https://github.com/hoytech/strfry.git
version: 1.0.4-1-g783f9ce8cc77


In [8]:
# List fields
print("=== List Fields ===")
print(f"supported_nips: {data['supported_nips']}")
print(f"relay_countries: {data['relay_countries']}")
print(f"language_tags: {data['language_tags']}")
print(f"tags: {data['tags']}")

=== List Fields ===
supported_nips: [1, 2, 4, 9, 11, 22, 28, 40, 70, 77]
relay_countries: None
language_tags: None
tags: None


## 3. Limitation Object

In [9]:
# Limitation dict
print("=== Limitation ===")
limitation = data['limitation']

if limitation:
    print(f"  max_message_length: {limitation.get('max_message_length')}")
    print(f"  max_subscriptions: {limitation.get('max_subscriptions')}")
    print(f"  max_limit: {limitation.get('max_limit')}")
    print(f"  auth_required: {limitation.get('auth_required')}")
    print(f"  payment_required: {limitation.get('payment_required')}")
else:
    print("  No limitations defined")

=== Limitation ===
  max_message_length: 1000000
  max_subscriptions: 300
  max_limit: 500
  auth_required: None
  payment_required: None


## 4. Full JSON Output

In [10]:
# All schema keys are present (with None for missing)
print("=== All Schema Keys ===")
print(f"Keys: {list(nip11.metadata.data.keys())}")
print()
print(json.dumps(nip11.metadata.data, indent=2, default=str))

=== All Schema Keys ===
Keys: ['name', 'description', 'banner', 'icon', 'pubkey', 'self', 'contact', 'supported_nips', 'software', 'version', 'privacy_policy', 'terms_of_service', 'limitation', 'retention', 'relay_countries', 'language_tags', 'tags', 'posting_policy', 'payments_url', 'fees']

{
  "name": "damus.io",
  "description": "Damus strfry relay",
  "banner": null,
  "icon": "https://damus.io/img/logo.png",
  "pubkey": "32e1827635450ebb3c5a7d12c1f8e7b2b514439ac10a67eef3d9fd9c5c68e245",
  "self": null,
  "contact": "jb55@jb55.com",
  "supported_nips": [
    1,
    2,
    4,
    9,
    11,
    22,
    28,
    40,
    70,
    77
  ],
  "software": "git+https://github.com/hoytech/strfry.git",
  "version": "1.0.4-1-g783f9ce8cc77",
  "privacy_policy": null,
  "terms_of_service": null,
  "limitation": {
    "max_message_length": 1000000,
    "max_subscriptions": 300,
    "max_limit": 500,
    "max_subid_length": null,
    "max_event_tags": null,
    "max_content_length": null,
    "min

## 5. Conversion to RelayMetadata

In [11]:
# Convert to RelayMetadata for DB storage
relay_metadata = nip11.to_relay_metadata()
print(f"RelayMetadata type: {type(relay_metadata).__name__}")
print(f"metadata_type: {relay_metadata.metadata_type}")
print(f"generated_at: {relay_metadata.generated_at}")
print(f"relay url: {relay_metadata.relay.url}")

RelayMetadata type: RelayMetadata
metadata_type: nip11
generated_at: 1767874234
relay url: wss://relay.damus.io


## 6. Fetch Multiple Relays

In [12]:
# Test multiple relays in parallel
relay_urls = [
    "wss://relay.damus.io",
    "wss://nos.lol",
    "wss://relay.nostr.band",
    "wss://nostr.wine",
    "wss://relay.snort.social",
]

async def fetch_nip11_safe(url: str) -> tuple[str, Nip11 | None, Exception | None]:
    relay = Relay(url)
    try:
        nip11 = await Nip11.fetch(relay, timeout=5.0)
        return url, nip11, None
    except Exception as e:
        return url, None, e

results = await asyncio.gather(*[fetch_nip11_safe(url) for url in relay_urls])

print("=== Multiple Relay Results ===")
for url, nip11_result, error in results:
    if nip11_result:
        d = nip11_result.metadata.data
        nips = d['supported_nips']
        nips_count = len(nips) if nips else 0
        print(f"✓ {url}: {d['name'] or 'No name'} - {nips_count} NIPs")
    else:
        print(f"✗ {url}: {type(error).__name__}")

=== Multiple Relay Results ===
✓ wss://relay.damus.io: damus.io - 10 NIPs
✓ wss://nos.lol: nos.lol - 10 NIPs
✓ wss://relay.nostr.band: Nostr.Band Relay - 8 NIPs
✓ wss://nostr.wine: nostr.wine - 10 NIPs
✓ wss://relay.snort.social: Snort - 10 NIPs


In [13]:
results[2][1].metadata.data

{'name': 'Nostr.Band Relay',
 'description': 'This is a fast relay with full archive of textual posts',
 'banner': None,
 'icon': None,
 'pubkey': '818a39b5f164235f86254b12ca586efccc1f95e98b45cb1c91c71dc5d9486dda',
 'self': None,
 'contact': 'mailto:admin@nostr.band',
 'supported_nips': [1, 11, 12, 15, 20, 33, 45, 50],
 'software': 'https://relay.nostr.band',
 'version': '0.4',
 'privacy_policy': None,
 'terms_of_service': None,
 'limitation': {'max_message_length': None,
  'max_subscriptions': None,
  'max_limit': None,
  'max_subid_length': None,
  'max_event_tags': None,
  'max_content_length': None,
  'min_pow_difficulty': None,
  'auth_required': None,
  'payment_required': None,
  'restricted_writes': None,
  'created_at_lower_limit': None,
  'created_at_upper_limit': None,
  'default_limit': None},
 'retention': None,
 'relay_countries': None,
 'language_tags': None,
 'tags': None,
 'posting_policy': None,
 'payments_url': None,
 'fees': {'admission': None, 'subscription': None,

## 7. Fetch with Proxy (Tor Relay)

To test Tor relays, you need a running Tor proxy (SOCKS5 on port 9050).

In [ ]:
# Example: Fetch from a Tor relay (requires Tor proxy running)
# Uncomment to test if you have Tor running locally

tor_relay = Relay("ws://oxtrdevav64z64yb7x6rjg3a4a7kblqcmjzreo5hsktyqhmpxsylzead.onion")
tor_proxy = "socks5://127.0.0.1:9050"

try:
    nip11_tor = await Nip11.fetch(tor_relay, proxy_url=tor_proxy, timeout=30.0)
    print(f"Tor relay name: {nip11_tor.metadata.data['name']}")
except Nip11FetchError as e:
    print(f"Tor fetch failed: {e.cause}")

print("Tor test skipped (uncomment to run with local Tor proxy)")

Tor fetch failed: [Errno 8] Couldn't connect to proxy tor:9050 [nodename nor servname provided, or not known]
Tor test skipped (uncomment to run with local Tor proxy)


## 8. Proxy Behavior

The `proxy_url` parameter behavior:
- **If provided**: Uses proxy regardless of relay network type
- **If not provided + clearnet relay**: Direct connection (works)
- **If not provided + overlay relay**: Fails (DNS can't resolve .onion/.i2p/.loki)

In [15]:
# Demonstrate: clearnet relay with proxy (works but unnecessary)
# clearnet_with_proxy = await Nip11.fetch(
#     Relay("wss://relay.damus.io"),
#     proxy_url="socks5://127.0.0.1:9050",  # Route through Tor
# )

# Demonstrate: overlay relay without proxy (will fail)
try:
    tor_relay = Relay("ws://example.onion")
    result = await Nip11.fetch(tor_relay, timeout=3.0)
    print("Unexpected success!")
except Nip11FetchError as e:
    print(f"Expected failure for overlay without proxy:")
    print(f"  Cause: {type(e.cause).__name__}")

Expected failure for overlay without proxy:
  Cause: ClientConnectorDNSError


## 9. Test Parsing with Synthetic Data

In [16]:
# Create Nip11 from synthetic data
test_relay = Relay("wss://test.relay.example")

synthetic_data = {
    "name": "Test Relay",
    "description": "A test relay for parsing",
    "supported_nips": [1, 2, 4, 9, 11, 40],
    "limitation": {
        "max_message_length": 128000,
        "auth_required": True,
    },
    "fees": {
        "admission": [{"amount": 21000, "unit": "sats"}],
    },
}

nip11_synthetic = Nip11(relay=test_relay, metadata=Metadata(synthetic_data))
d = nip11_synthetic.metadata.data

print("=== Synthetic Nip11 ===")
print(f"name: {d['name']}")
print(f"supported_nips: {d['supported_nips']}")
print(f"limitation: {d['limitation']}")
print(f"fees: {d['fees']}")

=== Synthetic Nip11 ===
name: Test Relay
supported_nips: [1, 2, 4, 9, 11, 40]
limitation: {'max_message_length': 128000, 'max_subscriptions': None, 'max_limit': None, 'max_subid_length': None, 'max_event_tags': None, 'max_content_length': None, 'min_pow_difficulty': None, 'auth_required': True, 'payment_required': None, 'restricted_writes': None, 'created_at_lower_limit': None, 'created_at_upper_limit': None, 'default_limit': None}
fees: {'admission': [{'amount': 21000, 'unit': 'sats'}], 'subscription': None, 'publication': None}


## 10. Parsing Edge Cases

In [17]:
# Test: Invalid types are filtered out
invalid_data = {
    "name": 12345,  # Should be string -> None
    "description": "Valid description",
    "supported_nips": [1, 2, "three", 4],  # Non-ints filtered
}

nip11_invalid = Nip11(relay=test_relay, metadata=Metadata(invalid_data))
d = nip11_invalid.metadata.data

print("=== Invalid Types Filtered ===")
print(f"name: {d['name']} (was int -> None)")
print(f"description: {d['description']}")
print(f"supported_nips: {d['supported_nips']} (filtered to [1, 2, 4])")

=== Invalid Types Filtered ===
name: None (was int -> None)
description: Valid description
supported_nips: [1, 2, 4] (filtered to [1, 2, 4])


In [18]:
# Test: Empty iterables become None
empty_data = {
    "name": "Empty Test",
    "supported_nips": [],  # Empty list -> None
    "limitation": {},  # Empty dict -> None
}

nip11_empty = Nip11(relay=test_relay, metadata=Metadata(empty_data))
d = nip11_empty.metadata.data

print("=== Empty Iterables ===")
print(f"supported_nips: {d['supported_nips']} (was [] -> None)")
print(f"limitation: {d['limitation']} (was {{}} -> None)")

=== Empty Iterables ===
supported_nips: None (was [] -> None)
limitation: {'max_message_length': None, 'max_subscriptions': None, 'max_limit': None, 'max_subid_length': None, 'max_event_tags': None, 'max_content_length': None, 'min_pow_difficulty': None, 'auth_required': None, 'payment_required': None, 'restricted_writes': None, 'created_at_lower_limit': None, 'created_at_upper_limit': None, 'default_limit': None} (was {} -> None)


## 11. Error Handling

In [19]:
# Test: Invalid relay URL
invalid_relay = Relay("wss://nonexistent.relay.invalid")

try:
    result = await Nip11.fetch(invalid_relay, timeout=3.0)
except Nip11FetchError as e:
    print("=== Nip11FetchError ===")
    print(f"relay: {e.relay.url}")
    print(f"cause: {type(e.cause).__name__}")

=== Nip11FetchError ===
relay: wss://nonexistent.relay.invalid
cause: ClientConnectorDNSError


In [20]:
# Test: Empty metadata raises ValueError
try:
    nip11_empty_fail = Nip11(relay=test_relay, metadata=Metadata({}))
except ValueError as e:
    print("=== Empty Metadata ===")
    print(f"Correctly raised: {e}")

=== Empty Metadata ===
Correctly raised: NIP-11 metadata cannot be empty (all values are None)


## 12. Class Defaults

In [21]:
print("=== Class Defaults ===")
print(f"_FETCH_TIMEOUT: {Nip11._FETCH_TIMEOUT} seconds")
print(f"_FETCH_MAX_SIZE: {Nip11._FETCH_MAX_SIZE} bytes ({Nip11._FETCH_MAX_SIZE // 1024} KB)")

=== Class Defaults ===
_FETCH_TIMEOUT: 10.0 seconds
_FETCH_MAX_SIZE: 65536 bytes (64 KB)


## Done!

### Key Points:
- Access fields via `nip11.metadata.data["key"]`
- All schema keys are present (with `None` for missing)
- Invalid types are silently converted to `None`
- Empty iterables become `None`
- Use `proxy_url` for Tor/I2P/Loki relays